In [1]:
from classes import MongoDBConnection,Embeddings
from pymongo.errors import CollectionInvalid,OperationFailure
connection = MongoDBConnection()
db = connection.get_database()
try:
    db.create_collection('azure')
except CollectionInvalid as e:
    print(e)
    print('Removing existing documents')
    db.azure.delete_many({})
    pass
collection = db.azure
embeddings = Embeddings()

collection azure already exists
Removing existing documents
Using provider:  azure_openai
Using model:  text-embedding-ada-002
Using dimensions:  1536


In [2]:
try:
    collection.create_search_index(
        {"definition":
            {
                "fields": [
                    {
                        "path":"embedding",
                        "numDimensions": 1536,
                        "similarity": "dotProduct",
                        "type": "vector"
                    }
                ]
            },
            "name": "vectorIndex",
            "type": "vectorSearch"
        }
    )
except OperationFailure as e:
    print(e)
    pass

Duplicate Index, full error: {'ok': 0.0, 'errmsg': 'Duplicate Index', 'code': 68, 'codeName': 'IndexAlreadyExists', '$clusterTime': {'clusterTime': Timestamp(1726569232, 1), 'signature': {'hash': b'$\xb6\xef\x81\x87\x1d\x00\x8c\xd0\xdc\xfbK\xeez\xaf\x8da:\x06@', 'keyId': 7377378677668970498}}, 'operationTime': Timestamp(1726569232, 1)}


In [3]:
collection.insert_one({'content':"Hello world!"})

InsertOneResult(ObjectId('66e95b1b4be0bf8d3b5097da'), acknowledged=True)

In [14]:
results = list(collection.aggregate([
    {
        "$vectorSearch":{
            "path":"embedding",
            "queryVector":embeddings.get_embedding("Hello bob!"),
            "numCandidates":10,
            "limit":10,
            "index":"vectorIndex"
        }
    },
    {
        "$project":{
            "content":1,
            "score":{
                "$meta":"vectorSearchScore"
            }
        }
    }
]))

for r in results:
    print(f"_id\t:\t{r["_id"]}\ncontent\t:\t{r['content']}\nscore\t:\t{r['score']}")
    print("-"*40)

_id	:	66e95b1b4be0bf8d3b5097da
content	:	Hello world!
score	:	0.930620551109314
----------------------------------------


In [ ]:
collection.insert_one({'content':"Greetings Bob!"})

In [ ]:
# collection.drop_search_index("vectorIndex")
# db.drop_collection("azure")